In [ ]:
# Teaser example: sort geological epochs
import jp_proxy_widget

# Here are geological epochs ordered from older to younger.
epoch_order = "archaen proterozoic paleozoic mesozoic tertiary pleistocene holocene".split()

# HTML with epochs out of order:
epoch_html = """ 
<ul>
    <li data-sort-id="paleozoic">paleozoic</li>
    <li data-sort-id="pleistocene">pleistocene</li>
    <li data-sort-id="archaen">archaen</li>
    <li data-sort-id="holocene">holocene</li>
    <li data-sort-id="mesozoic">mesozoic</li>
    <li data-sort-id="tertiary">tertiary</li>
    <li data-sort-id="proterozoic">proterozoic</li>
</ul>
"""

from IPython.display import HTML
HTML(epoch_html)

In [ ]:
def sort_epochs_widget():
    "Make a widget for sorting the epochs."
    sort_epochs = jp_proxy_widget.JSProxyWidget()

    # Use jQueryUI (sortable):
    sort_epochs.check_jquery()

    # Execute javascript to show the HTML in the widget.
    sort_epochs.js_init("""
        // empty out the widget element.
        element.empty()
        // Append a title area for feedback.
        element.title = $("<b>Please drag the epochs to the correct order.</b>").appendTo(element);
        // Append a divider.
        $("<hr>").appendTo(element);
        // Add the initially incorrectly ordered html.
        element.epoch_list = $(epoch_html).width("200px").appendTo(element);
        // Make the epoch list sortable
        element.epoch_list.sortable()
    """, epoch_html=epoch_html)
    return sort_epochs

sort_epochs_no_callbacks = sort_epochs_widget()

# display the widget
sort_epochs_no_callbacks

In [ ]:
# Make a new widget and add more functionality:
sort_epochs = sort_epochs_widget()

# Use this list to store the displayed order
displayed_order = []

# Define a function to report whether the displayed order is correct
def check_order(order):
    "Python callback: check the displayed order sent from the Widget and report results."
    # save the order globally for debugging and reference.
    displayed_order[:] = order
    ok = True
    for (index, identifier) in enumerate(order):
        correct_index = epoch_order.index(identifier)
        color = "yellow"
        if correct_index < index:
            color = "cyan"
            ok = False
        elif correct_index > index:
            color = "magenta"
            ok = False
        # colorize the element based on correct/incorrect order
        sort_epochs.element.set_color(identifier, color)
    if ok:
        sort_epochs.element.set_title("The order is correct!")
    else:
        sort_epochs.element.set_title("Cyan elements should be earlier and magenta elements later.")

sort_epochs.js_init("""
        // Attach a function to change the background color of a list element.
        element.set_color = function(id, color) {
            // jQuery magic:
            element.find("[data-sort-id=" + id + "]").css("background-color", color)
        };
        // Attach a function to change the title
        element.set_title = function(text) {
            element.title.html("<b>" + text + "</b>");
        };
        // Attach a function to report the displayed order back to the Python callback check_order:
        element.report_order = function() {
            check_order(element.epoch_list.sortable("toArray", {"attribute": "data-sort-id"}))
        };
        // Add a check button linked to the Python callback
        $('<button type="button">Check it</button>').click(element.report_order).appendTo(element);
""", 
                    # Bind the python callback
                    check_order=check_order)

sort_epochs

In [ ]:
sort_epochs.element.set_title("goober")
sort_epochs.element.set_color("paleozoic", "pink")

In [ ]:
displayed_order

In [ ]:
sort_epochs.print_status()